## get Cause efffect phrase from the sentence.

In [1]:
from openai import OpenAI


In [18]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test_cleaned.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'
#depbertTEST_CEextract 

pm_DB = pd.read_csv('data/in/KNN_RAG/depbertTEST_CEextract.csv')

#start#comparison with DEBERT paper table
#pm_DB = pd.read_csv('data/in/KNN_RAG/sem_CE_test_cleaned.csv')
#pm_DB = pm_DB[107:]
#pm_DB.reset_index(drop=True, inplace=True)

##end

instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

142

In [19]:

pm_DB

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings
0,107,158,158,158,158,"Finally, Slone's fear of AIDS and the mental...",stab,fear,"[-0.00929092988371849, -0.00021748989820480347..."
1,108,159,159,159,159,The suffering caused by the embargo is som...,embargo,suffering,"[-0.009257652796804905, -0.031496670097112656,..."
2,109,160,160,160,160,Sorace was unaware that her anger was caused...,abuse,anger,"[0.01198310311883688, -0.010999979451298714, 0..."
3,110,161,161,161,161,The idea of Roman expansion into North Africa ...,power,fear,"[0.017029738053679466, -0.010211342945694923, ..."
4,111,162,162,162,162,Tides are caused by the gravitational pull ...,gravitational pull,Tides,"[0.010173842310905457, -0.015142759308218956, ..."
...,...,...,...,...,...,...,...,...,...
137,244,321,321,321,321,"The winds are caused by moisture , originat...",moisture,winds,"[-0.0030969225335866213, -0.015087406150996685..."
138,245,322,322,322,322,The drag caused by the Earth's atmosphere ...,atmosphere,drag,"[-0.00888124480843544, -0.003990810364484787, ..."
139,246,323,323,323,323,Most of the flooding has been caused by the ...,clogging,flooding,"[-0.01667359657585621, 0.0030442047864198685, ..."
140,247,324,324,324,324,The dips are caused by the thermal diffuse o...,y-rays,dips,"[-0.018423471599817276, -0.0059008412063121796..."


In [20]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

In [21]:
semeval_data = semeval_data.dropna(subset=['causality_phrase'])
semeval_data['causality_phrase'] = semeval_data['causality_phrase'].str.lower()#LOWERCASE

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

In [29]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')
###model#gpt-3.5-turbo

#+ fewshot_examples
def C_E_CP_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause": "", "effect": "", "causality_phrase": ""}.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
                   
          Below are around 20 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=200,
      top_p=1
    )
    
    return response.choices[0].message.content


In [30]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_example_sentence, fewshot_example_causalityPhrase

In [31]:
# semeval
filtered_rows = pm_DB

In [32]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [33]:
#!pip install fuzzywuzzy

In [34]:
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

cause_list = []
effect_list = []
causality_phrase_list = []
fewshotfound_inputsentence_index=[]
for i in range (len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    
    
    
    #KNN+RAG few shot examples augmentation. #lowercase sent
    fewshot_index = get_fewshot_examples_index(input_sentence.lower())#LOWERCASE###########FEWSHOT   
    
    fewshot_sentence_rag, fewshot_patterns_rag = get_fewshotexamples_C_E_CP(fewshot_index) #############fewshot
    #fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    print('input_sentence : ', input_sentence)
    ###
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_sentence_knn = [examples[j] for j in most_similar_indices]
    fewshot_patterns_knn = [causality_phrase[p] for p in most_similar_indices]
    
    fewshot_sentence_comb = fewshot_sentence_rag+fewshot_sentence_knn
    fewshot_patterns_comb = fewshot_patterns_rag+fewshot_patterns_knn

    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_patterns_comb[i]} 
                    for i, sentence in enumerate(fewshot_sentence_comb)}
    
    fewshot_examples = json.dumps(fewshot_examples)   

    #print('fewshot_comb : ', fewshot_examples)
    
    #####end
    
    # output generation
    out = C_E_CP_Identifier(input_sentence, fewshot_examples)
    
    print('out: ', out)
    
    json_out = json.loads(out)
    cause = json_out['cause']
    effect = json_out['effect']
    causality_phrases = json_out['causality_phrase']
    
    
    cause_list.append(cause)
    effect_list.append(effect)
    causality_phrase_list.append(causality_phrases)
    
    #print('input_sentence: ', input_sentence )
    #print('fewshot_examples: ', fewshot_examples )
    #print('cause: ', cause)
    #print('effect: ', effect)
    #print('causality_phrase: ', causality_phrases)
    
    
    print(i)
    #if i==0:
        #break;
    #print(causality_identifier)
    
    

input_sentence :  Finally, Slone's  fear  of AIDS and the mental distress she suffered from this fear were caused by the needle  stab .
out:  {"cause": "needle stab", "effect": "fear of AIDS and the mental distress", "causality_phrase": "were caused by"}
0
input_sentence :  The  suffering  caused by the  embargo  is sometimes reported here as well.
out:  {"cause": "embargo", "effect": "suffering", "causality_phrase": "caused by"}
1
input_sentence :  Sorace was unaware that her  anger  was caused by the  abuse .
out:  {"cause": "abuse", "effect": "anger", "causality_phrase": "was caused by"}
2
input_sentence :  The idea of Roman expansion into North Africa started with the  fear  and jealousy caused by the great economic  power  of Carthage.
out:  {"cause": "fear and jealousy", "effect": "great economic power of Carthage", "causality_phrase": "caused by"}
3
input_sentence :   Tides  are caused by the  gravitational pull  of the moon and sun, and the rotation of the Earth.
out:  {"cause"

out:  {"cause": "some", "effect": "human growth hormone", "causality_phrase": "described by"}
33
input_sentence :  The  election  was caused by the  appointment  of Donald Sumner, formerly Conservative MP for Orpington, to be a county court judge.
out:  {"cause": "appointment of Donald Sumner, formerly Conservative MP for Orpington", "effect": "election", "causality_phrase": "was caused by"}
34
input_sentence :  This was our first day back volunteering there but he wasn't feeling good, had a  fever  from  immunizations .
out:  {"cause": "immunizations", "effect": "fever", "causality_phrase": "from"}
35
input_sentence :  The  acidity  is caused by the gradual buildup of  carbon dioxide  (CO2) in the atmosphere, dissolving into the oceans.
out:  {"cause": "gradual buildup of carbon dioxide (CO2) in the atmosphere", "effect": "acidity", "causality_phrase": "is caused by"}
36
input_sentence :  The straight, blue ion  tail  is caused by the  solar wind  and the curved dust tail is caused by

input_sentence :  The full amount paid to Aardman for the  blaze , which was triggered by an electrical  fault , was not disclosed.
out:  {"cause": "electrical fault", "effect": "blaze", "causality_phrase": "triggered by"}
67
input_sentence :  Two of my girls had the high  fever  from  vaccines , and our family doctor thought nothing of it.
out:  {"cause": "vaccines", "effect": "high fever", "causality_phrase": "from"}
68
input_sentence :   Acne  is caused by  oily skin  and often has black or whiteheads, inflamed sores, and infected sores along with scars.
out:  {"cause": "oily skin", "effect": "acne", "causality_phrase": "is caused by"}
69
input_sentence :  The  rainbows  are caused by the  refraction , or bending, of sunlight as it passes through the raindrops.
out:  {"cause": "refraction", "effect": "rainbows", "causality_phrase": "are caused by"}
70
input_sentence :  The response to the  harm  caused by the  flood , although institutionally organized, relied mostly on individuals 

out:  {"cause": "Israeli blockade", "effect": "crisis", "causality_phrase": "caused by"}
100
input_sentence :  The truck carried homemade weapons, and the  blast  was caused by the  mishandling  of weapons.
out:  {"cause": "mishandling of weapons", "effect": "blast", "causality_phrase": "was caused by"}
101
input_sentence :  Thursday afternoon the staff from the University of Wisconsin-Madison near Lake Mendota felt a shake and heard a  noise  like a boom which was triggered by an ice  quake .
out:  {"cause": "ice quake", "effect": "noise like a boom", "causality_phrase": "triggered by"}
102
input_sentence :  Chronic  coughing  from  smoking  increases the risk of developing a hernia.
out:  {"cause": "smoking", "effect": "chronic coughing", "causality_phrase": "from"}
103
input_sentence :  The  delays  are caused by local  governments  trying to wring more money out of the feder.
out:  {"cause": "local governments", "effect": "delays", "causality_phrase": "are caused by"}
104
input_sen

input_sentence :  However, recent research demonstrates that most  ulcers  of the duodenum are caused by an  infectious agent  named Helicobacter pylori or H. pylori.
out:  {"cause": "infectious agent", "effect": "ulcers of the duodenum", "causality_phrase": "are caused by"}
133
input_sentence :  To alleviate the  inconvenience  caused by the  fence , the Cabinet approved a plan to construct 11 passages through the barrier to facilitate movement in and out of the city.
out:  {"cause": "fence", "effect": "inconvenience", "causality_phrase": "caused by"}
134
input_sentence :  The  error  was caused by the illegal embedded code  tags .
out:  {"cause": "illegal embedded code tags", "effect": "error", "causality_phrase": "was caused by"}
135
input_sentence :  The poet Essex Hemphill conquered  sorrow  after the  loss  of a friend by taking up the cause of that friend.
out:  {"cause": "loss of a friend", "effect": "sorrow", "causality_phrase": "after the"}
136
input_sentence :  The  winds  a

In [35]:
len(filtered_rows)

142

In [36]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [37]:
filtered_rows['cause'] = cause_list
filtered_rows['effect'] = effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase
0,107,158,158,158,158,"Finally, Slone's fear of AIDS and the mental...",stab,fear,"[-0.00929092988371849, -0.00021748989820480347...",needle stab,fear of AIDS and the mental distress,were caused by
1,108,159,159,159,159,The suffering caused by the embargo is som...,embargo,suffering,"[-0.009257652796804905, -0.031496670097112656,...",embargo,suffering,caused by
2,109,160,160,160,160,Sorace was unaware that her anger was caused...,abuse,anger,"[0.01198310311883688, -0.010999979451298714, 0...",abuse,anger,was caused by
3,110,161,161,161,161,The idea of Roman expansion into North Africa ...,power,fear,"[0.017029738053679466, -0.010211342945694923, ...",fear and jealousy,great economic power of Carthage,caused by
4,111,162,162,162,162,Tides are caused by the gravitational pull ...,gravitational pull,Tides,"[0.010173842310905457, -0.015142759308218956, ...",gravitational pull of the moon and sun,tides,are caused by
...,...,...,...,...,...,...,...,...,...,...,...,...
137,244,321,321,321,321,"The winds are caused by moisture , originat...",moisture,winds,"[-0.0030969225335866213, -0.015087406150996685...",moisture,winds,are caused by
138,245,322,322,322,322,The drag caused by the Earth's atmosphere ...,atmosphere,drag,"[-0.00888124480843544, -0.003990810364484787, ...",Earth's atmosphere,drag,caused by
139,246,323,323,323,323,Most of the flooding has been caused by the ...,clogging,flooding,"[-0.01667359657585621, 0.0030442047864198685, ...",clogging of drains,flooding,has been caused by
140,247,324,324,324,324,The dips are caused by the thermal diffuse o...,y-rays,dips,"[-0.018423471599817276, -0.0059008412063121796...",thermal diffuse of y-rays,dips,are caused by


In [38]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [39]:
#get accuracy
from fuzzywuzzy import fuzz

def get_accuracy(filtered_rows):
    prediction= []
    e_prediction=[]
    c_prediction=[]
    for i in range(len(filtered_rows)):
        #print(i)
        #lower case..
        cause_pred = filtered_rows['cause'][i].lower()
        cause_t = filtered_rows['cause_t'][i].lower()
        
        effect_pred = filtered_rows['effect'][i].lower()
        effect_t = filtered_rows['effect_t'][i].lower()

        # Slide through input_sentence with a window size of 8
        #print('effect_pred',  effect_pred)
        #print('effect_t',  effect_t)


        score = get_score(cause_t, cause_pred)################ non -medcause
        #score = fuzz.ratio(cause_pred, cause_t)################medcause
        if score >90:################90 non -medcause
            pred_c = 1
            c_prediction.append(pred_c)
        else:
            pred_c = 0
            c_prediction.append(pred_c)
            
        score = get_score(effect_t, effect_pred)################non - medcause
        #score = fuzz.ratio(effect_pred, effect_t)################medcause

        if score >90: ################90 non -medcause
            pred_e = 1
            e_prediction.append(pred_e)
        else:
            pred_e = 0
            e_prediction.append(pred_e)

        
        if ((pred_e ==1) & (pred_c ==  1)):
            prediction.append(1)
        else:
            prediction.append(0)
            
    

    return prediction, c_prediction, e_prediction


In [40]:
prediction, c_prediction, e_prediction = get_accuracy(filtered_rows)

prediction length lower ulcer p: indigestion
prediction length lower shiver p: flicker
prediction length lower some p: hormone
prediction length lower becky p: company
prediction length lower heat p: burns
prediction length lower fear p: robbers


In [41]:
sum(prediction)/len(filtered_rows)

0.9014084507042254

In [ ]:
len(prediction)

In [ ]:
sum(c_prediction)/len(filtered_rows)

In [ ]:
sum(e_prediction)/len(filtered_rows)

In [ ]:
filtered_rows['predictions'] = prediction

In [ ]:
filtered_rows

In [ ]:
#filtered_rows.to_csv('data/out/RAG_causality/task2FIN/GPT4_KP_sem_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [ ]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#filtered_rows.to_csv('data/out/RAG_causality/CE_extracts/randomshotCOMB_ADE_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/rand_ADE_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [ ]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
#task2FIN